### import

In [1]:
import os
import re
import datetime
import unicodedata
import importlib.util
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException, MoveTargetOutOfBoundsException

### Import local

In [2]:
with open('../common/logger.py') as f:
    exec(f.read())

# logger: defined in logger.py, use this logger to log all the process of genai with prefix 'genai'.
logger = get_logger(name='genai')

In [3]:
# Imports a module from another directory. 
# Ex: get_module('common', 'web_driver.py') will import web_driver.py from common folder and return it as a module object. 
def get_module(folder_name, file_name):
    module_name = file_name.split('.')[0]
    module_path = os.path.join(os.getcwd(), '..', folder_name, file_name)
    spec = importlib.util.spec_from_file_location(module_name, module_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

In [4]:
driver_module = get_module('common', 'web_driver.py')
get_driver = driver_module.get_driver

variable_module = get_module('internal', 'variable.py')
USERNAME = variable_module.GENAI_USERNAME
PASSWORD = variable_module.GENAI_PASSWORD

### Handle text

In [5]:
# normalize remove non ASCII chars
def normalize_text(text):
    normalized_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    return normalized_text

# replace non-word characters with space
def clean_text(text):
    cleaned_text = re.sub(r'[^\w\s]', ' ', text)
    return cleaned_text

# remove paths and urls from text 
def remove_paths_and_urls(text):
    pattern = r'(?:[A-Za-z]:\\[^\\\n]*|\/[^\/\n]*)+|http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # replace url with space
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

def handle_text(text):
    text = remove_paths_and_urls(text)
    text = normalize_text(text)
    text = clean_text(text)
    line_text = '.  '.join(text.splitlines())
    return line_text

### Genai class

In [6]:
class Genai:
    def __init__(self): 
        self.driver = get_driver()
        self.driver.get('https://genai.sec.samsung.net/summarize')

        self.login() 
        self.driver.implicitly_wait(50)
        sleep(50)

        self.IS_CLOSE_POPUP = False
        self.close_popup()
        self.close_template()
        logger.info('Init genai success.')
    
    def login(self):
        try:
            user_ip = self.driver.find_element(By.ID, 'userNameInput')
            password_ip = self.driver.find_element(By.ID, 'passwordInput')
            login_btn = self.driver.find_element(By.ID, 'submitButton')

            user_ip.send_keys(USERNAME)
            password_ip.send_keys(PASSWORD)
            login_btn.click()
        except Exception as e:
            logger.error(f'Login fail: {type(e).__name__}')
            self.cap_screen(type(e).__name__)
    
    def click_close_btn(self):
        close_btns = self.driver.find_elements(By.XPATH, "//*[text()='Close']")
        if len(close_btns) > 0:
            logger.info('Close tutorial popup.')
            for btn in close_btns:
                try:            
                    self.driver.execute_script("arguments[0].click();", btn)              
                except Exception:
                    logger.info("Clicked on close button using js.")

        btn_by_class = self.driver.find_elements(By.CLASS_NAME, 'v-btn__content')
        if len(btn_by_class) > 0:
            logger.info('Close notice popup.')
            for btn in btn_by_class:
                try:
                    if (btn.text == 'Close'):
                        btn.click()
                except Exception:
                    pass
    
    def close_popup(self):
        self.click_close_btn()
        try:
            # move to last image -> close tutorial
            circle_btn = self.driver.find_elements(By.CLASS_NAME, "mdi-circle")
            if (len(circle_btn)  == 0):
                return
            circle_btn[4].click()

            close_btn = self.driver.find_elements(By.CLASS_NAME, "mdi-close")
            close_btn[1].click()
        except Exception as e:
            logger.error(f'Close popup fail: {type(e).__name__}')
            self.cap_screen(type(e).__name__)

    def close_template(self):
        # close hover mouse: move mouse to another position
        action = ActionChains(self.driver)
        action.move_by_offset(0, 0).perform()

        try: 
            logger.info('Close prompt template.')
            img_close_Prompt_template = self.driver.find_element(By.XPATH, '//img[@alt="drawerClose"]')
            img_close_Prompt_template.click()
        except:
            pass

    # create new chat and send prompt
    def search(self, text, wait = 20, is_fail = False):
        if self.IS_CLOSE_POPUP == False:
            self.close_popup()

        max_wait = 50
        text = handle_text(text)
        if len(text) < 20:
            return text
        
        try:
            new_chat_btn = self.driver.find_element(By.CLASS_NAME, "new-chat-floating")
            new_chat_btn.click()

            search_promt = "summarize below text in english focus on samsung keyword(required English):"
            search_input = search_promt + text 
            input_txt = self.driver.find_element(By.ID, "input-41")
            input_txt.send_keys(search_input)
            send_btn = self.driver.find_element(By.CLASS_NAME, "inquiry")
            send_btn.click()
            sleep(wait)

            elements = self.driver.find_elements(By.TAG_NAME, "P")
            if len(elements) == 0:
                logger.error('Summary fail: No result found. Please check your input text.')
                self.cap_screen()
            else:
                self.IS_CLOSE_POPUP = True
                result  = elements[-1].text
                return result
        # login expried -> relogin genai
        except (ElementClickInterceptedException, MoveTargetOutOfBoundsException) as e:
            if is_fail == False:
                self.__init__()
                return self.search(text= text, wait= 20, is_fail=True)
            else:
                self.cap_screen(type(e).__name__)
        # other exception -> retry search
        except Exception as e:
            if is_fail == False:
                return self.search(text, wait = max_wait, is_fail=True)
            else:
                logger.error(f"Search fail: {e}")
                self.cap_screen(type(e).__name__)

        return "Fail"
    
    def cap_screen(self, name = "ScreenShot"):
        current_time = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
        screenshot_path = f'../output/screenshot/{current_time}_{name}.png'
        self.driver.save_screenshot(screenshot_path)
        print(f'Screen capture saved at{screenshot_path}.')

    def __del__(self):
        self.driver.quit()